In [229]:
%reset
import numpy as np
import pandas as pd
from collections import defaultdict

In [230]:
class KNN:
	def __init__(self, k=1):
		self.K = k

	def train(self, x, y):
		self.X = x
		self.Y = y

	def predict(self, x_pred):

		scores = []
		for x in self.X:
			# ['Taken_Courses', 'Grades', 'CGPA', 'SGPA', 'Warning', 'Recommended']
			if len(set(x[0])&set(x_pred[0])) != 0:
				score = 0
				uncommon = len(set(x[0])) + len(set(x_pred[0])) - len(set(x[0])&set(x_pred[0]))*2
				common = list(set(x[0])&set(x_pred[0]))
				
				for c in common:
					id1 = x_pred[0].index(c)
					id2 = x[0].index(c)
					score += (x_pred[1][id1] - x[1][id2])**2
				score **=0.5
				score += ((x_pred[2][0] - x[2][0])**2)
				score += ((x_pred[3][0] - x[3][0])**2)
				score += ((x_pred[4][0] - x[4][0])**2)
				score += uncommon
				scores.append(score)
			else:
				scores.append(999)
			
		scores = np.array(scores)	
		min_vals = np.argpartition(scores,self.K)[:self.K]
		l = [j for sub in list(self.Y[:, 0][min_vals]) for j in sub]
		d = dict((x, l.count(x)) for x in set(l))
		d = {k: v for k, v in sorted(d.items(),reverse= True, key=lambda item: item[1])}
		return d
	def evaluate(self):
		pass


In [231]:
# makes a list of values for same keys in diff dictionaries
def combine_dict(list_of_dictionaries):
    dic = defaultdict(lambda: list())
    for d in list_of_dictionaries:  
        for item in d.items():
            dic[item[0]].append(item[1])
    return dic

# Scale Data features on range or SD
def scale(df,col,on_range = True):
    if on_range: # Scaling based on Range
        df[col] -= df[col].min()
        df[col] /= df[col].max()
    else: # Scaling based on SD
        sd = df[col].std()
        df[col] -= df[col].mean()
        df[col] /= sd
    return


In [232]:
def EE_roll_nos(df):
    EE_Crs=['Engineering Workshop','Engineering Drawing','Engineering Workshop',
            'Electrical Network Analysis','Signals and Systems','Electro Mechanical Systems',
            'Multivariable Calculus','µP Interfacing and Programming','Electromagnetic Theory',
            'Feedback Control Systems','Engineering Economics','Applied Calculus',
            'Physics for Engineers','Linear Circuit Analysis','Electronic Devices and Circuits',
            'Probability and Random']
    EE_Depart=pd.DataFrame(columns = df.columns.to_list())
    for crs in EE_Crs:
        EE_Depart=EE_Depart.append(df[df['Course Title'] == crs],ignore_index=True)
    roll_no = list(EE_Depart['Sr. No'].unique())
    return roll_no
def Business_roll_nos(df):
    Business_Crs=['IT in Business','Fundamentals of Accounting','Business Math - I',
                  'Business Math - II','Financial Accounting','Business Statistics',
                  'Management Accounting','Statistical Inference','Statistical Inference',
                  'Business Communication - I','Organizational Behavior','Business Finance',
                  'Consumer Behavior','Micro Economics','Financial Management',
                  'Operations Management','Human Resource Management','Macro Economics',
                  'Management Info. Systems','Business Law','Methods in Business Research',
                  'Financial Institutions and Markets','Business Communication - II',
                  'Strategic Management','Business Ethics']
    Business_Depart=pd.DataFrame(columns = df.columns.to_list())
    for crs in Business_Crs:
        Business_Depart=Business_Depart.append(df[df['Course Title'] == crs],ignore_index=True)
    roll_no = list(Business_Depart['Sr. No'].unique())
    return roll_no
def CS_roll_nos(df):
    CS_Crs=['Applied Physics','Basic Electronics','Calculus - I','Calculus - II',
            'Data Structures','Discrete Structures','Comp. Organization and Assembly Lang.',
            'Database Systems','Operating Systems','Design and Analysis of Algorithms',
            'Theory of Automata','Computer Networks','Object Oriented Analysis and Design',
            'Probability and Statistics','Software Engineering','Artificial Intelligence',
            'Human Computer Interaction','Computer Architecture','Professional Issues in IT']
    CS_Depart=pd.DataFrame(columns = df.columns.to_list())
    for crs in CS_Crs:
        CS_Depart=CS_Depart.append(df[df['Course Title'] == crs],ignore_index=True)
    roll_no = list(CS_Depart['Sr. No'].unique())
    return roll_no

In [233]:
def convert_to_feature_labels(data):
    cols = data.columns.to_list()
    students = data.index.to_list()

    xyData = pd.DataFrame(columns=['Taken_Courses','Grades','CGPA','SGPA','Warning','Recommended'])

    for student in students:
        student_data = data.loc[student]
        for i in range(len(data.loc[student].Semester)-1):
            # print(student_data['Course Title'])
            xyData = xyData.append({'Taken_Courses':student_data['Course'][i],
            'Grades':student_data.Grade_Point[i],
            'CGPA': student_data.CGPA[i],
            'SGPA':student_data.SGPA[i],
            'Warning':student_data.Warning[i],
            'Recommended':student_data['Course'][i+1]} ,
            ignore_index=True)
            if len(student_data['Course'][i]) != len(student_data.Grade_Point[i]):
                print(len(student_data['Course'][i]),len(student_data.Grade_Point[i]))

    return xyData

In [234]:
def convert_to_single_line(roll_nos, semesters, Stud_Data):
    modified_data = pd.DataFrame(columns=['Roll_No'],data=roll_nos)
    modified_data.set_index(['Roll_No'],inplace=True)

    for c in ['Course','Grade_Point','Semester','Warning','SGPA','CGPA']:
        modified_data[c] = np.empty((len(roll_nos), 0)).tolist()

    roll_nos.sort()


    for roll in roll_nos:
        for semester in semesters:
            try:
                # Convert Each Group To a Separate DataFrame
                df = pd.DataFrame(Stud_Data.loc[roll])
                df = df.groupby(['Semester']).get_group(semester)

                # Transfer Data to a new Dataframe
                modified_data.loc[roll].Semester.append([semester])
                modified_data.loc[roll].Course.append(list(df['Course Title'].to_list()))
                modified_data.loc[roll].Grade_Point.append(list(df['Grade Point'].to_list()))
                modified_data.loc[roll].Warning.append(list([df['Warning'].to_list()[0]]))
                modified_data.loc[roll].SGPA.append(list([df['SGPA'].to_list()[0]]))
                modified_data.loc[roll].CGPA.append(list([df['CGPA'].to_list()[0]]))
            except KeyError:
                pass
    return modified_data

In [244]:
def adviseMe(k , x_train , y_train,semesterData, course_guide ):
    classifier = KNN(k)
    classifier.train(x_train,y_train)
    courses = classifier.predict(semesterData)
    creditLimit = 19
    if semesterData[4][0] != 0: # Check Warning Count
        creditLimit = 15
    assignedLimit = 0
    coursesList = list(courses.keys())
    suggestedCourses = []
    otherSuggestions = []
    for c in coursesList:
        if assignedLimit + course_guide.get(c) <= creditLimit:
            suggestedCourses.append(c)
            assignedLimit += course_guide.get(c)
        elif assignedLimit >= creditLimit:
            otherSuggestions.append(c)
    return suggestedCourses,assignedLimit,otherSuggestions

In [236]:
Stud_Data = pd.read_excel('./Course_Advisory_Data.xlsx')
Stud_Data.dropna(inplace= True)
# Extract Col, Semester, RollNo of Students

# Dictionary for Course Code and Course Title and Credit Hours
course_guide = dict(zip(Stud_Data['Course Title'], Stud_Data['Credit Hours']))
# Dictionary for Grade and Grade Point
grade_dictionary = dict(zip(Stud_Data['Grade Point'], Stud_Data['Grade']))


In [237]:
delete_courses = ['Advanced Research Methods','Research in Marketing','MS Thesis - I',
        'MS Thesis - II','PhD Thesis - I','PhD Thesis - II','PhD Thesis - III',
        'PhD Thesis - IV','Applied Programming','Research Methodology']

delete_semesters =   [20111,20121,20123,20131]


for line in delete_courses:
    indexNames = Stud_Data[ Stud_Data['Course Title'] == line ].index
    Stud_Data.drop(indexNames,inplace=True)
for ds in delete_semesters:
    indexNames = Stud_Data[ Stud_Data['Semester'] == ds ].index
    Stud_Data.drop(indexNames,inplace=True)
grades = [ 'I', 'W', 'FA', 'L1', 'LL', 'L2', 'F/R', 'CN']
assignedVal = -1
for g in grades:
    Stud_Data.loc[Stud_Data['Grade'] == g, 'Grade Point'] = assignedVal
    assignedVal -= 1

In [238]:
# Student Roll No Department Wise
# Electrical Engineering
DEE = EE_roll_nos(Stud_Data)
# Computer Science
DCS = CS_roll_nos(Stud_Data)
# School of Management
DBM = Business_roll_nos(Stud_Data)

# Set Index To Student's Roll No's
Stud_Data.set_index('Sr. No',inplace= True)
columns = Stud_Data.columns.to_list()
semesters = list(Stud_Data.Semester.unique())
roll_nos = list(Stud_Data.index.unique())

In [239]:
# Scale Numerical Cols on Range
# scale(data, 'CGPA')
# scale(data, 'SGPA')
# scale(data, 'Grade Point')

In [240]:

semesters = ['Fall 2016','Spring 2017','Summer 2017',
'Fall 2017','Spring 2018','Summer 2018','Fall 2018',
'Spring 2019','Summer 2019','Fall 2019']


modified_data_EE = convert_to_single_line(DEE, semesters, Stud_Data)

modified_data_CS = convert_to_single_line(DCS, semesters, Stud_Data)

modified_data_BM = convert_to_single_line(DBM, semesters, Stud_Data)

# Write Data to an excel Stud_Datae
# modified_data.to_pickle('intermediateData.sav')

In [241]:
# data = pd.read_pickle('intermediateData.sav')

xyData_EE = convert_to_feature_labels(modified_data_EE)

xyData_CS = convert_to_feature_labels(modified_data_CS)

xyData_BM = convert_to_feature_labels(modified_data_BM)

# xyData.to_pickle('xydata.xlsx')

In [271]:
xyData = xyData_CS.sample(frac = 1).reset_index(drop=True)

ratio = 0.8
size = int(len(xyData) * ratio)
x_train = xyData.values[:size,:5]
y_train = xyData.values[:size,5:]

# print(x_train.shape,y_train.shape)

x_test = xyData.values[size:,:5]
y_test = xyData.values[size:,5:]

# print(x_test.shape,y_test.shape)


In [279]:
r = 455

cour, lim, other = adviseMe(7 ,x_train ,y_train, x_test[r], course_guide )

for idx,y in enumerate(x_test[r][0]):
    print(y,grade_dictionary.get(x_test[r][1][idx]))

print('\nWarning Count : ',x_test[r][4][0])
print('\n\n')

print(*cour, '\nCredit Hours : '+str(lim), sep= '\n')

print('\n\n\n*********Other Possible Suggestions****************\n' ,*other, sep= '\n')


Pakistan Studies C-
Data Structures B-
Fundamentals of Management B
Comp. Organization and Assembly Lang. Lab C-
Discrete Structures C+
Comp. Organization and Assembly Lang. B-

Warning Count :  0



Technical and Business Writing
Principles of Marketing
Database Systems
Operating Systems
Operating Systems Lab
Computer Architecture
Database Systems Lab

Credit Hours : 17



*********Other Possible Suggestions****************



In [280]:
pre_req = pd.read_excel('./Prerequisites of All Courses.xlsx')
pre_req

,Sr#,Code,Title,Type,Cr.Hrs,Equivalence,Status,Pre Req.
0,1,CL103,Computer Programming Lab,Lab,1.0,397,Active,"CL101, CS101"
1,2,CL203,Database Systems Lab,Lab,1.0,726,Active,CS201
2,3,CL205,Operating Systems Lab,Lab,1.0,727,Active,CS201
3,4,CL210,Data Structures and Algorithms - Lab,Lab,1.0,1010,Active,"CS214, CL214"
4,5,CL214,Programming Fundamentals - Lab,Lab,1.0,397,Active,"EL116, EE116"
...,...,...,...,...,...,...,...,...
1315,1316,CS480,Network Programming Lab,Lab,3.0,43,Dormant,NaN
1316,1317,CS639,Computer Aided Modeling and Simulation,Taught Course,3.0,1203,Active,NaN
1317,1318,SE505,Model Based Testing,Taught Course,3.0,1169,Active,NaN
1318,1319,CS110,Computer Programming,Taught Course,3.0,208,Dormant,NaN


In [281]:
course_guide

{'Basic Electronics': 3,
 'Introduction to Computing Lab': 1,
 'Introduction to Computing': 3,
 'Islamic and Religious Studies': 3,
 'English Language': 3,
 'Calculus - I': 3,
 'English Language Lab': 1,
 'Computer Programming Lab': 1,
 'Digital Logic Design - Lab': 1,
 'Computer Programming': 3,
 'Pakistan Studies': 3,
 'Calculus - II': 3,
 'Digital Logic Design': 3,
 'English Composition': 3,
 'Comp. Organization and Assembly Lang. Lab': 1,
 'Business Communication - I': 3,
 'Comp. Organization and Assembly Lang.': 3,
 'Linear Algebra': 3,
 'Data Structures': 3,
 'Discrete Structures': 3,
 'Operating Systems': 3,
 'Operating Systems Lab': 1,
 'Psychology': 3,
 'Probability and Statistics': 4,
 'Theory of Automata': 3,
 'Computing Methods - I': 3,
 'Object Oriented Analysis and Design': 3,
 'Object Oriented Analysis and Design Lab': 1,
 'Computer Networks Lab': 1,
 'Computer Architecture': 3,
 'Fundamentals of Management': 3,
 'Computer Networks': 3,
 'Advanced Programming': 3,
 'Data